In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import preprocessing
text, labels = preprocessing.preprocessing("data/Twitter_Data.csv")

training_portion = .8
divide = int(len(text)*training_portion)
train_tweets, validation_tweets = text[:divide], text[divide:]
train_labels, validation_labels = labels[:divide], labels[divide:]
    

vocab_size = 18000
max_length = 50
import tokenize_data
train_padded, validation_padded = tokenize_data.tokenize(train_tweets, validation_tweets, max_length, vocab_size)
training_label, validation_label = tokenize_data.label_tokenize(train_labels, validation_labels)

print(train_padded.shape)
print(validation_padded.shape)

(13525, 50)
(3382, 50)


In [3]:
import embedding
embedding_dim = 200
embeddings_matrix = embedding.embedding(train_tweets, vocab_size, embedding_dim)

In [4]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length = max_length, weights=[embedding_matrix]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation = 'sigmoid'),
    tf.keras.layers.Dense(4, activation = 'softmax')
])

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = tf.keras.optimizers.RMSprop(), metrics=['acc'])
model.summary()

NameError: name 'embedding_matrix' is not defined

In [ ]:
num_epochs = 5
history = model.fit(train_padded, train_lables_padded, epochs = num_epochs, validation_data=(validation_padded, padded_validation_label_seq), verbose = 2)

In [ ]:
plot_graphs(history, "acc")
plot_graphs(history, "loss")